In [1]:
%pip install scipy
%pip install statsmodels
%pip install ace-tools
%pip install pdfkit
%pip install imgkit
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import statsmodels.api as sm
import pdfkit
from PIL import Image
import imgkit




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load the dataset
file_path = "AP PW1 Data Cleaned.csv" #REPLACE WITH YOUR OWN PATH
df = pd.read_csv(file_path)

# Rename the first column to 'Date'
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

# Load market return dataset
market_file_path = "Market Return Cleaned.csv" #REPLACE WITH YOUR OWN PATH
market_df = pd.read_csv(market_file_path)

market_df.rename(columns={market_df.columns[0]: 'Date'}, inplace=True)

# Merge datasets on Date
merged_df = pd.merge(df, market_df, on='Date')

merged_df

,Date,ME1 BM1,ME1 BM2,ME1 BM3,ME1 BM4,ME1 BM5,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM3,ME4 BM4,ME4 BM5,ME5 BM1,ME5 BM2,ME5 BM3,ME5 BM4,ME5 BM5,Market Return,Rf
0,196307,1.1307,-0.3091,0.7079,0.1062,-1.3211,-1.8071,0.1899,-1.0105,-1.9644,...,-1.9184,-1.5741,-1.8567,0.1547,0.4833,1.2286,-0.5862,-1.1026,-0.66,0.27
1,196308,4.2370,1.3834,1.4977,2.3755,4.7567,5.5665,4.5191,4.4424,4.4188,...,6.2332,7.6782,5.3469,5.7691,4.2550,4.5936,8.2831,6.3824,4.82,0.25
2,196309,-2.8878,0.6263,-1.0204,-1.6000,-0.4320,-4.0502,-1.5034,-0.8798,-1.1812,...,-1.7800,-3.9639,-1.9943,-1.3595,-0.8054,-0.8135,-0.2145,-3.4963,-1.84,0.27
3,196310,1.2885,-0.7071,1.3132,0.0855,2.3988,1.1916,4.2342,2.3524,2.2012,...,2.6229,4.8492,0.6113,5.3339,1.7427,-0.2469,2.3915,0.4857,2.24,0.29
4,196311,-3.3751,-3.7534,-1.8055,-1.0487,-1.0538,-4.2561,-1.7534,-0.7809,-0.1002,...,-0.7933,1.3610,3.5388,-1.2556,1.0072,-1.7425,-2.0838,1.3455,-1.12,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,202402,9.7415,6.8190,11.7272,2.6284,4.2753,9.6645,6.6236,7.1860,4.3233,...,3.7097,4.6091,2.5429,6.0699,1.5096,8.0769,4.9935,2.9089,4.65,0.42
728,202403,-3.2823,1.5975,0.7470,1.6994,4.7100,0.1016,2.4109,2.5531,2.8258,...,5.1387,8.1868,7.6653,1.6713,6.2897,3.6674,6.1407,8.9863,2.40,0.43
729,202404,-12.5428,-7.1861,-7.5104,-7.4424,-7.5468,-6.8279,-8.2722,-6.3410,-5.6108,...,-6.0023,-6.8561,-5.7337,-3.9996,1.1908,-4.4722,-4.9938,-5.4745,-5.14,0.47
730,202405,7.2793,3.2521,4.9308,3.5565,4.7890,4.5944,3.2302,5.5056,4.8427,...,2.3887,5.2926,3.6067,5.9812,4.1988,3.0522,4.0484,4.2659,3.90,0.44


note: market return = market excess return

### Question 1a

In [3]:
# Drop the 'Date' column for calculations
data = merged_df.drop(columns=['Date'])

# Extract size and B/M quintiles from column names
column_names = [col.split() for col in data.columns if col not in ['Market Return', 'Rf']]
size_quintile_labels = [name[0] for name in column_names]
bm_quintile_labels = [name[1] if len(name) > 1 else '' for name in column_names]

# Compute arithmetic average monthly returns
average_returns = data.drop(columns=['Market Return', 'Rf']).mean()
average_returns_df = pd.DataFrame(average_returns, columns=['Avg Monthly Return'])
average_returns_df['Size Quintile'] = size_quintile_labels
average_returns_df['B/M Quintile'] = bm_quintile_labels


# Pivot the table to have size in rows and B/M quintiles in columns
average_returns_pivot = average_returns_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Avg Monthly Return')

# Use 'Market Return' for beta calculations
#market_returns = data['Market Return']
#betas = {}

#for col in data.columns:
    #if col not in ['Market Return', 'Rf']:  # Avoid regressing market on itself
      #  X = market_returns
      #  X = sm.add_constant(X)
      #  y = data[col] - data['Rf']
      #  model = sm.OLS(y, X).fit()
      #  betas[col] = model.params[1]  # Extract beta coefficient

# Convert results to DataFrame
#betas_df = pd.DataFrame.from_dict(betas, orient='index', columns=['Market Beta'])
#betas_df['Size Quintile'] = size_quintile_labels
#betas_df['B/M Quintile'] = bm_quintile_labels

# Pivot the table for betas
#betas_pivot = betas_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Market Beta')


In [16]:
### Compute Betas using Excess Returns (Q1a) ###
market_returns = data['Market Return']

betas = {}
for col in data.columns:
    if col not in ['Market Return', 'Rf']:
        portfolio_excess_return = data[col] - data['Rf']
        print(portfolio_excess_return)
        y = portfolio_excess_return  # Excess portfolio return
        X = market_returns
        X = sm.add_constant(X)  # Excess market return
        model = sm.OLS(y, X).fit()
        betas[col] = model.params[1]

betas_df = pd.DataFrame.from_dict(betas, orient='index', columns=['Market Beta'])
betas_df['Size Quintile'] = size_quintile_labels
betas_df['B/M Quintile'] = bm_quintile_labels
betas_pivot = betas_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Market Beta')

0       0.8607
1       3.9870
2      -3.1578
3       0.9985
4      -3.6451
        ...   
727     9.3215
728    -3.7123
729   -13.0128
730     6.8393
731    -4.7089
Length: 732, dtype: float64
0     -0.5791
1      1.1334
2      0.3563
3     -0.9971
4     -4.0234
        ...  
727    6.3990
728    1.1675
729   -7.6561
730    2.8121
731   -2.7302
Length: 732, dtype: float64
0       0.4379
1       1.2477
2      -1.2904
3       1.0232
4      -2.0755
        ...   
727    11.3072
728     0.3170
729    -7.9804
730     4.4908
731    -6.0055
Length: 732, dtype: float64
0     -0.1638
1      2.1255
2     -1.8700
3     -0.2045
4     -1.3187
        ...  
727    2.2084
728    1.2694
729   -7.9124
730    3.1165
731   -2.0509
Length: 732, dtype: float64
0     -1.5911
1      4.5067
2     -0.7020
3      2.1088
4     -1.3238
        ...  
727    3.8553
728    4.2800
729   -8.0168
730    4.3490
731   -2.9688
Length: 732, dtype: float64
0     -2.0771
1      5.3165
2     -4.3202
3      0.9016
4     -4.526

/tmp/ipykernel_2676/797774951.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[col] = model.params[1]
/tmp/ipykernel_2676/797774951.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[col] = model.params[1]
/tmp/ipykernel_2676/797774951.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas[col] = model.params[1]
/tmp/ipykernel_2676/797774951.py:13: FutureWarning: Series.__getitem__ treating keys as positions is

In [15]:
print("Market Betas")
betas_pivot

Market Betas


B/M Quintile,BM1,BM2,BM3,BM4,BM5
Size Quintile,,,,,
ME1,1.403966,1.214194,1.102264,1.030033,1.062842
ME2,1.370574,1.173920,1.068489,1.012050,1.139560
ME3,1.294861,1.114825,1.005997,0.995592,1.078712
ME4,1.191362,1.074139,1.003058,0.990179,1.072261
ME5,0.979966,0.912620,0.854242,0.879778,0.973967


In [6]:
print("\nAverage Returns:")
average_returns_pivot.round(2)


Average Returns:


B/M Quintile,BM1,BM2,BM3,BM4,BM5
Size Quintile,,,,,
ME1,0.63,1.12,1.13,1.32,1.45
ME2,0.88,1.14,1.22,1.25,1.36
ME3,0.90,1.16,1.11,1.23,1.34
ME4,1.01,1.02,1.07,1.19,1.23
ME5,0.97,0.92,0.95,0.89,1.05


## Question 1b

In [7]:
# Compute time-series standard deviations of portfolio returns
std_devs = data.drop(columns=['Market Return', 'Rf']).std()
std_devs_df = pd.DataFrame(std_devs, columns=['Time-Series Std Dev'])
std_devs_df['Size Quintile'] = size_quintile_labels
std_devs_df['B/M Quintile'] = bm_quintile_labels
std_devs_pivot = std_devs_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Time-Series Std Dev')

# Compute Sharpe Ratios using the new risk-free rate column
sharpe_ratios = (average_returns - data['Rf'].mean()) / std_devs
sharpe_ratios_df = pd.DataFrame(sharpe_ratios, columns=['Sharpe Ratio'])
sharpe_ratios_df['Size Quintile'] = size_quintile_labels
sharpe_ratios_df['B/M Quintile'] = bm_quintile_labels
sharpe_ratios_pivot = sharpe_ratios_df.pivot(index='Size Quintile', columns='B/M Quintile', values='Sharpe Ratio')



In [8]:
print("\nTime-Series Standard Deviations:")
std_devs_pivot.round(2)


Time-Series Standard Deviations:


B/M Quintile,BM1,BM2,BM3,BM4,BM5
Size Quintile,,,,,
ME1,8.01,6.96,6.09,5.85,6.30
ME2,7.19,6.09,5.56,5.39,6.27
ME3,6.56,5.54,5.08,5.18,5.95
ME4,5.84,5.18,5.01,5.10,5.84
ME5,4.70,4.41,4.34,4.77,5.68


In [9]:
print("\nSharpe Ratios:")
sharpe_ratios_pivot.round(2)


Sharpe Ratios:


B/M Quintile,BM1,BM2,BM3,BM4,BM5
Size Quintile,,,,,
ME1,0.03,0.11,0.13,0.16,0.17
ME2,0.07,0.13,0.15,0.16,0.16
ME3,0.08,0.14,0.15,0.17,0.17
ME4,0.11,0.13,0.14,0.16,0.15
ME5,0.13,0.13,0.14,0.11,0.12


## Question 1c

In [10]:
### Compute T-Statistics for Portfolio Returns ###
t_stats = (average_returns / (std_devs / np.sqrt(len(data))))
t_stats_df = pd.DataFrame(t_stats, columns=['T-Statistic'])
t_stats_df['Size Quintile'] = size_quintile_labels
t_stats_df['B/M Quintile'] = bm_quintile_labels
t_stats_pivot = t_stats_df.pivot(index='Size Quintile', columns='B/M Quintile', values='T-Statistic')

### Compute T-Statistics for Quintile 5 Minus 1 ###
t_stats_5_minus_1 = {}

# Compute for each B/M quintile (ME5 - ME1)
for bm in set(bm_quintile_labels):
    me1_col = [col for col in data.columns if f'ME1 {bm}' in col][0]
    me5_col = [col for col in data.columns if f'ME5 {bm}' in col][0]
    diff = data[me5_col] - data[me1_col]
    t_stats_5_minus_1[f'ME5 {bm} - ME1 {bm}'] = diff.mean() / (diff.std() / np.sqrt(len(data)))

# Compute for each Size quintile (BM5 - BM1)
for me in set(size_quintile_labels):
    bm1_col = [col for col in data.columns if f'{me} BM1' in col][0]
    bm5_col = [col for col in data.columns if f'{me} BM5' in col][0]
    diff = data[bm5_col] - data[bm1_col]
    t_stats_5_minus_1[f'{me} BM5 - {me} BM1'] = diff.mean() / (diff.std() / np.sqrt(len(data)))

# Compute for ME5 BM5 - ME1 BM1
top_col = [col for col in data.columns if 'ME5 BM5' in col][0]
bottom_col = [col for col in data.columns if 'ME1 BM1' in col][0]
diff = data[top_col] - data[bottom_col]
t_stats_5_minus_1["ME5 BM5 - ME1 BM1"]= diff.mean() / (diff.std() / np.sqrt(len(data)))


# Convert results to DataFrame
t_stats_5_minus_1_df = pd.DataFrame(t_stats_5_minus_1, index=['Quintile 5 - 1']).T

# Append to the main t-stats table
t_stats_pivot = pd.concat([t_stats_pivot, t_stats_5_minus_1_df.T])





In [11]:
t_stats_5_minus_1_df

,Quintile 5 - 1
ME5 BM2 - ME1 BM2,-1.029730
ME5 BM3 - ME1 BM3,-1.048958
ME5 BM4 - ME1 BM4,-2.627936
ME5 BM5 - ME1 BM5,-2.236672
ME5 BM1 - ME1 BM1,1.575946
ME2 BM5 - ME2 BM1,3.135356
ME4 BM5 - ME4 BM1,1.353518
ME3 BM5 - ME3 BM1,2.748239
ME5 BM5 - ME5 BM1,0.476181
ME1 BM5 - ME1 BM1,5.114709


In [12]:
#cross checking ME5BM5-ME1BM1 test stat
five_minus_one = merged_df["ME5 BM5"] - merged_df["ME1 BM1"]
t_statistic, p_value = stats.ttest_1samp(five_minus_one, 0)
print("T-statistic:", t_statistic)


T-statistic: 1.6862683612250056


In [13]:
print("\n Test Statistics:")
t_stats_pivot.round(2)


 Test Statistics:


,BM1,BM2,BM3,BM4,BM5,ME5 BM2 - ME1 BM2,ME5 BM3 - ME1 BM3,ME5 BM4 - ME1 BM4,ME5 BM5 - ME1 BM5,ME5 BM1 - ME1 BM1,ME2 BM5 - ME2 BM1,ME4 BM5 - ME4 BM1,ME3 BM5 - ME3 BM1,ME5 BM5 - ME5 BM1,ME1 BM5 - ME1 BM1,ME5 BM5 - ME1 BM1
ME1,2.12,4.36,5.01,6.09,6.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ME2,3.30,5.05,5.92,6.27,5.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ME3,3.71,5.66,5.90,6.45,6.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ME4,4.68,5.30,5.80,6.34,5.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ME5,5.57,5.67,5.94,5.04,4.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quintile 5 - 1,NaN,NaN,NaN,NaN,NaN,-1.03,-1.05,-2.63,-2.24,1.58,3.14,1.35,2.75,0.48,5.11,1.69


In [14]:
def save_dataframe_as_image(df, filename, title):
    fig, ax = plt.subplots(figsize=(10, 6))  # Adjust figure size
    ax.set_title(title, fontsize=14, fontweight='bold', pad=5)  # Reduce space between title and table
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, rowLabels=df.index, cellLoc='center', loc='center')
    
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

# Save different tables as images with titles
save_dataframe_as_image(average_returns_pivot, "average_returns_table.png", "Arithmetic Average Monthly Returns")
save_dataframe_as_image(std_devs_pivot, "std_devs_table.png", "Time-Series Standard Deviations")
save_dataframe_as_image(sharpe_ratios_pivot, "sharpe_ratios_table.png", "Sharpe Ratios")
save_dataframe_as_image(t_stats_pivot, "t_statistics_table.png", "T-Statistics")

